In [1]:
import nltk
#nltk.download()
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import re, string, random
#from nltk import FreqDist, classify, NaiveBayesClassifier

[nltk_data] Downloading package punkt to /Users/ryan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ryan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ryan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /Users/ryan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')

In [3]:
print(positive_tweets[0]) #just to show what is being stored in positive/negative/text

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)


In [4]:
print(text[0])

RT @KirkKus: Indirect cost of the UK being in the EU is estimated to be costing Britain £170 billion per year! #BetterOffOut #UKIP


In [5]:
#Removes hyperlinks and other special symbols then groups elements of tokenized words together using lemmatization 
def cleanNoise(tokenized, stopWords = ()):
    cleaned = []
    for ele, mark in pos_tag(tokenized):
        ele = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', ele)
        ele = re.sub("(@[A-Za-z0-9_]+)","", ele)
        if mark.startswith("NN"):
            grp = 'n'
        elif mark.startswith('VB'):
            grp = 'v'
        else:
            grp = 'a'
            
         #lemmatizes  
        ele = WordNetLemmatizer().lemmatize(ele,grp)
        
        #Checks to make sure ele is not empty, not a repeat, and not punctuation
        if len(ele)>0:
            if ele not in string.punctuation:
                if ele.lower() not in cleaned:
                    cleaned.append(ele.lower())
    return cleaned

In [6]:
def finalClean(tweets):
    cleanedTweets = []
    for x in tweets: 
        clean = cleanNoise(word_tokenize(x), stopWords)
        cleanedTweets.append(clean)

    stop_words = get_stop_words('en')
    remove = ['rt', 's', '\'s', '\'\'','\'','http', 'of', 'with', 'be', 'much', 'their', 'the', 'boston', 'college', 'bostoncollege', 'bc']
    i = 0
    while i<3:
        for line in cleanedTweets:
            for each in line: 
                if each in stop_words:
                    line.remove(each)
                elif each in remove:
                    line.remove(each)
                elif each.startswith('//'):
                    line.remove(each)
        i= i+1
        
    return cleanedTweets

In [7]:
stopWords = stopwords.words('english')
test = twitter_samples.tokenized('positive_tweets.json')[0]
print(test)
print(cleanNoise(test,stopWords))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']
['#followfriday', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


In [8]:
def modelIn(cleanInput):
    for x in cleanInput:
        yield dict([y, True] for y in x)

In [9]:
def modelMaker():
    posTweets = twitter_samples.strings('positive_tweets.json')
    negTweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')
    stopWords = stopwords.words('english')
    
    posTokens = twitter_samples.tokenized('positive_tweets.json')
    negTokens = twitter_samples.tokenized('negative_tweets.json')
    posCleanTokens = []
    negCleanTokens = []
    
    for x in posTokens:
        clean = cleanNoise(x,stopWords)
        posCleanTokens.append(clean)
    for x in negTokens:
        clean = cleanNoise(x,stopWords)
        negCleanTokens.append(x)
        
    #posTrue = {}
    #negTrue = {}
    #for x in posCleanTokens:
     #   posTrue.append([x, True])
    #for x in negCleanTokens:
     #   negTrue.append([x,True])
    posTrue = modelIn(posCleanTokens)
    negTrue = modelIn(negCleanTokens)
    
    posData = []
    negData = []
    
    for x in posTrue:
        posData.append((x, "Positive"))
    for x in negTrue:
        negData.append((x,"Negative"))
    
    posNegData = posData + negData
    
    #print(posNegData[:5])
    random.shuffle(posNegData)

    train_data = posNegData[:7000]
    test_data = posNegData[7000:]
    
    #right now the model uses Naive bayes but if we are going to do collapsed gibbs we will replace 
    #This part and we will also probably have to reformat the input
    classifier = nltk.NaiveBayesClassifier.train(train_data)
    return(classifier)

In [10]:
words = ['be','good','saw']
dict([x,True] for x in words)

{'be': True, 'good': True, 'saw': True}

In [11]:
model = modelMaker()

In [12]:
custom_tweet = "I love sunshine and rainbows"
custom_tokens = cleanNoise(word_tokenize(custom_tweet))
print(custom_tweet, model.classify(dict([token, True] for token in custom_tokens)))

I love sunshine and rainbows Positive


In [13]:
import pandas as pd
import tweepy
from textblob import TextBlob
from gensim import corpora, models
import gensim
from stop_words import get_stop_words

/Users/ryan/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [14]:

auth = tweepy.OAuthHandler('API KEY', 'API KEY')
auth.set_access_token('API KEY', 'API KEY')
api = tweepy.API(auth)

In [15]:
def get_tweets(hashtags, date, numtweets):

    tagged_tweets = tweepy.Cursor(api.search, q=hashtags, lang="en",since=date, tweet_mode="extended").items(numtweets)

    tweets_df = pd.DataFrame(columns=('Date and Time', 'id', 'source', 'Geo', 'Place', 'Retweets', 'tweet', 'Sentiment Polarity'))
    list_tweets = []
    i = 1
    for tweet in tagged_tweets:

        list_tweets.append(tweet.full_text)
        
        tweets_df.at[i, 'id'] = tweet.id
        tweets_df.at[i, 'source'] = tweet.source
        tweets_df.at[i, 'tweet'] = tweet.full_text
        tweets_df.at[i, 'Geo'] = tweet.geo
        tweets_df.at[i, 'Place'] = tweet.place
        tweets_df.at[i, 'Favorites'] = tweet.favorite_count
        tweets_df.at[i, 'Retweets'] = tweet.retweet_count
        tweets_df.at[i, 'Date and Time'] = tweet.created_at
        analysis = TextBlob(tweet.full_text)
        tweets_df.at[i, 'Sentiment Polarity'] = analysis.sentiment.polarity
        i = i + 1

    tweets_df.to_csv(hashtags+'tweets.csv', encoding='utf-8')
    #print(tweets_df.head())
    #print(list_tweets[0])
    return (list_tweets)

In [16]:
tweets = get_tweets('bostoncollege', '2020-04-01', 1000)

In [17]:
pos = 0
neg = 0
for x in tweets:
    custom_tokens = cleanNoise(word_tokenize(x))
    res = model.classify(dict([token, True] for token in custom_tokens))
    if res == 'Positive':
        pos+=1
    else:
        neg +=1
print(pos,neg)

445 27


In [18]:
testTweet = tweets[0]
print(testTweet)
custom_tokens = cleanNoise(word_tokenize(testTweet))
print(custom_tokens, model.classify(dict([token, True] for token in custom_tokens)))

Looking for a psychology, sociology, social work or human services internship for the fall? RSVP to attend an info session on May 25 for our #Call2Talk mental health, emotional support &amp; suicide prevention program.
cc: @holy_cross @BostonCollege @bclynchschool @BU_CCD @UML_CSP https://t.co/Uu17EcGh0w
['looking', 'for', 'a', 'psychology', 'sociology', 'social', 'work', 'or', 'human', 'service', 'internship', 'the', 'fall', 'rsvp', 'to', 'attend', 'an', 'info', 'session', 'on', 'may', '25', 'our', 'call2talk', 'mental', 'health', 'emotional', 'support', 'amp', 'suicide', 'prevention', 'program', 'cc', 'holy_cross', 'bostoncollege', 'bclynchschool', 'bu_ccd', 'uml_csp', 'http', '//t.co/uu17ecgh0w'] Positive


In [19]:

##Training LDA on Tweets to get Topics

cleanedTweets = finalClean(tweets)
dictionary = corpora.Dictionary(cleanedTweets)
corpus = [dictionary.doc2bow(text) for text in cleanedTweets]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word = dictionary, passes=100)

print(ldamodel.print_topics())

[(0, '0.018*"can" + 0.014*"vaccine" + 0.013*"make" + 0.013*"less" + 0.013*"immunity" + 0.013*"herd" + 0.013*"prof." + 0.013*"hindering" + 0.013*"u.s." + 0.013*"fill"'), (1, '0.011*"today" + 0.010*"can" + 0.010*"“" + 0.010*"alumni" + 0.010*"us" + 0.009*"nydailynews" + 0.009*"bcalumni" + 0.009*"offer" + 0.009*"mask" + 0.009*"like"'), (2, '0.014*"fossil" + 0.014*"industry" + 0.014*"invest" + 0.014*"fuel" + 0.013*"despite" + 0.013*"pope" + 0.010*"repeated" + 0.010*"call" + 0.010*"sthubertknights" + 0.010*"stnorbert"'), (3, '0.013*"work" + 0.008*"community" + 0.008*"team" + 0.008*"member" + 0.008*"look" + 0.007*"virtual" + 0.007*"’" + 0.007*"event" + 0.007*"live" + 0.007*"us"'), (4, '0.016*"good" + 0.014*"’" + 0.013*"``" + 0.011*"world" + 0.011*"1950s–1980s" + 0.011*"arab" + 0.011*"abstraction" + 0.011*"barjeelart" + 0.010*"shape" + 0.010*"taking"'), (5, '0.019*"bulletinbc" + 0.015*"go" + 0.010*"commitment" + 0.010*"jaeden" + 0.010*"chipola" + 0.010*"zackery" + 0.010*"next" + 0.009*"late" +

In [20]:
##Get new tweets, get topic probability distribution
#tweets_new = get_tweets('bostoncollege', '2020-04-01', 200)
#cleanedTweets2 = finalClean(tweets_new)
#dictionary2 = corpora.Dictionary(cleanedTweets)
#corpus2 = [dictionary.doc2bow(text) for text in cleanedTweets2]

#newTweet = corpus2[0]
#print(cleanedTweets2[0])
#print(cleanedTweets[0])
tweetsWithTopics = []
for i in range (len(corpus)):
    print(cleanedTweets[i])
    vector = ldamodel[corpus[i]]
    tweetsWithTopics.append(vector)
    print(vector)
print(tweetsWithTopics)


['looking', 'psychology', 'sociology', 'social', 'work', 'human', 'service', 'internship', 'fall', 'rsvp', 'attend', 'info', 'session', 'may', '25', 'call2talk', 'mental', 'health', 'emotional', 'support', 'amp', 'suicide', 'prevention', 'program', 'cc', 'holy_cross', 'bclynchschool', 'bu_ccd', 'uml_csp']
[(8, 0.9699938)]
['’', 'late', 'join', 'glasswingvc', 'ignite', 'fellows', 'mit', 'northeastern', 'tuftsuniversity', 'amp', 'insightful', 'discussion', '“', 'strengthening', 'campus', 'startup', 'ecosystems', '”', 'today', '6:00', 'pm', 'et', 'clubhouse', 'innovation', 'university']
[(6, 0.96538085)]
['proud', 'member', 'mendell', 'school', 'community', 'since', '2013-14', 'year', 'delight', 'julia', 'receive', 'prestigious', 'honor', 'bclynchschool']
[(4, 0.93998617)]
['bcbookmarks', 'professor', 'gerald', 'easter', 'travel', 'writer', 'mara_vorhees', 'tell', 'story', 'russian', 'royalty', 'sunken', 'treasu…']
[(9, 0.9357099)]
['bulletinbc', 'relief', 'way', 'new', 'acc', 'commission

In [21]:
topic0 = []
topic1 = []
topic2 = []
topic3 = []
topic4 = []
topic5 = []
topic6 = []
topic7 = []
topic8 = []
topic9 = []

for topicProbs in tweetsWithTopics:
    maxProb = 0
    maxTopic = 0
    for probs in topicProbs:
        topic, prob = probs
        if prob > maxProb:
            maxProb = prob
            maxTopic = topic
    
    if maxTopic == 0:
        topic0.append(cleanedTweets.pop())
    elif maxTopic == 1:
        topic1.append(cleanedTweets.pop())
    elif maxTopic == 2:
        topic2.append(cleanedTweets.pop())
    elif maxTopic == 3:
        topic3.append(cleanedTweets.pop())
    elif maxTopic == 4:
        topic4.append(cleanedTweets.pop())
    elif maxTopic == 5:
        topic5.append(cleanedTweets.pop())
    elif maxTopic == 6:
        topic6.append(cleanedTweets.pop())
    elif maxTopic == 7:
        topic7.append(cleanedTweets.pop())
    elif maxTopic == 8:
        topic8.append(cleanedTweets.pop())
    elif maxTopic == 9:
        topic9.append(cleanedTweets.pop())

In [23]:
#Topic 0
topic0Sent = []
for i in range (len(topic0)):
    topic0Sent.append(model.classify(dict([token, True] for token in topic0[i])))
topic0numPos = 0
topic0numNeg = 0
for i in topic0Sent:
    if i[0] == 'P':
        topic0numPos = topic0numPos + 1
    else:
        topic0numNeg = topic0numNeg + 1
    
print("Percent pos = ", topic0numPos/len(topic0Sent))
print(topic0numPos)
print(topic0numNeg)

#Topic 1
topic1Sent = []
for i in range (len(topic1)):
    topic1Sent.append(model.classify(dict([token, True] for token in topic1[i])))
topic1numPos = 0
topic1numNeg = 0
for i in topic1Sent:
    if i[0] == 'P':
        topic1numPos = topic1numPos + 1
    else:
        topic1numNeg = topic1numNeg + 1
    
print("Percent pos = ", topic1numPos/len(topic1Sent))
print(topic1numPos)
print(topic1numNeg)

#Topic 2
topic2Sent = []
for i in range (len(topic2)):
    topic2Sent.append(model.classify(dict([token, True] for token in topic2[i])))
topic2numPos = 0
topic2numNeg = 0
for i in topic2Sent:
    if i[0] == 'P':
        topic2numPos = topic2numPos + 1
    else:
        topic2numNeg = topic2numNeg + 1
    
print("Percent pos = ", topic2numPos/len(topic2Sent))
print(topic2numPos)
print(topic2numNeg)

#Topic 3
topic3Sent = []
for i in range (len(topic3)):
    topic3Sent.append(model.classify(dict([token, True] for token in topic3[i])))
topic3numPos = 0
topic3numNeg = 0
for i in topic3Sent:
    if i[0] == 'P':
        topic3numPos = topic3numPos + 1
    else:
        topic3numNeg = topic3numNeg + 1
    
print("Percent pos = ", topic3numPos/len(topic3Sent))
print(topic3numPos)
print(topic3numNeg)

#Topic 4
topic4Sent = []
for i in range (len(topic4)):
    topic4Sent.append(model.classify(dict([token, True] for token in topic4[i])))
topic4numPos = 0
topic4numNeg = 0
for i in topic4Sent:
    if i[0] == 'P':
        topic4numPos = topic4numPos + 1
    else:
        topic4numNeg = topic4numNeg + 1
    
print("Percent pos = ", topic4numPos/len(topic4Sent))
print(topic4numPos)
print(topic4numNeg)

#Topic 5
topic5Sent = []
for i in range (len(topic5)):
    topic5Sent.append(model.classify(dict([token, True] for token in topic5[i])))
topic5numPos = 0
topic5numNeg = 0
for i in topic5Sent:
    if i[0] == 'P':
        topic5numPos = topic5numPos + 1
    else:
        topic5numNeg = topic5numNeg + 1
    
print("Percent pos = ", topic5numPos/len(topic5Sent))
print(topic5numPos)
print(topic5numNeg)

#Topic 6
topic6Sent = []
for i in range (len(topic6)):
    topic6Sent.append(model.classify(dict([token, True] for token in topic6[i])))
topic6numPos = 0
topic6numNeg = 0
for i in topic6Sent:
    if i[0] == 'P':
        topic6numPos = topic6numPos + 1
    else:
        topic6numNeg = topic6numNeg + 1
    
print("Percent pos = ", topic6numPos/len(topic6Sent))
print(topic6numPos)
print(topic6numNeg)

#Topic 7
topic7Sent = []
for i in range (len(topic7)):
    topic7Sent.append(model.classify(dict([token, True] for token in topic7[i])))
topic7numPos = 0
topic7numNeg = 0
for i in topic7Sent:
    if i[0] == 'P':
        topic7numPos = topic7numPos + 1
    else:
        topic7numNeg = topic7numNeg + 1
    
print("Percent pos = ", topic7numPos/len(topic7Sent))
print(topic7numPos)
print(topic7numNeg)

#Topic 8
topic8Sent = []
for i in range (len(topic8)):
    topic8Sent.append(model.classify(dict([token, True] for token in topic8[i])))
topic8numPos = 0
topic8numNeg = 0
for i in topic8Sent:
    if i[0] == 'P':
        topic8numPos = topic8numPos + 1
    else:
        topic8numNeg = topic8numNeg + 1
    
print("Percent pos = ", topic8numPos/len(topic8Sent))
print(topic8numPos)
print(topic8numNeg)

#Topic 9
topic9Sent = []
for i in range (len(topic9)):
    topic9Sent.append(model.classify(dict([token, True] for token in topic9[i])))
topic9numPos = 0
topic9numNeg = 0
for i in topic9Sent:
    if i[0] == 'P':
        topic9numPos = topic9numPos + 1
    else:
        topic9numNeg = topic9numNeg + 1
    
print("Percent pos = ", topic9numPos/len(topic9Sent))
print(topic9numPos)
print(topic9numNeg)


Percent pos =  0.92
46
4
Percent pos =  0.8536585365853658
35
6
Percent pos =  0.8771929824561403
50
7
Percent pos =  0.9512195121951219
39
2
Percent pos =  0.9285714285714286
52
4
Percent pos =  0.9285714285714286
39
3
Percent pos =  0.9302325581395349
40
3
Percent pos =  0.9555555555555556
43
2
Percent pos =  0.8108108108108109
30
7
Percent pos =  0.8666666666666667
52
8
